# Scratch for Maze environment

https://medium.com/swlh/fun-with-python-1-maze-generator-931639b4fb7e


In [165]:
from random import random, randint
import numpy as np

In [203]:
cell = 'c'
wall = 'w'

def init_maze(width, height):
    maze = []
    for i in range(0, height):
        line = []
        for j in range(0, width):
            line.append('u')
        maze.append(line)
    return maze

In [274]:
height = 4
width = 10
maze = init_maze(width, height)
print_maze(maze)

u u u u u u u u u u 

u u u u u u u u u u 

u u u u u u u u u u 

u u u u u u u u u u 



In [275]:

def print_maze(maze):
    for i in range(0, len(maze)):
        for j in range(0, len(maze[0])):
            if maze[i][j] == 'u':
                print(f'{maze[i][j]}', end=" ")
            elif maze[i][j] == 'c':
                print(f'{maze[i][j]}', end=" ")
            else:
                print(f'{maze[i][j]}', end=" ")
        print('\n')

In [276]:
print_maze(maze)

u u u u u u u u u u 

u u u u u u u u u u 

u u u u u u u u u u 

u u u u u u u u u u 



In [282]:
# Pick a starting spot to build maze
starting_height = int(random()*height)
starting_width = int(random()*width)
starting_height, starting_width


(0, 2)

In [283]:
# We need to make sure that we do not start on a block that is on the edge of the maze
if starting_height == 0:
    starting_height += 1
    print("1")
if starting_height > height-1:
    starting_height -= 1
    print("2")
if starting_width == 0:
    starting_width += 1
    print("3")
if starting_width > width-1:
    starting_width -= 1
    print("4")
# maze = init_maze(width, height)
starting_height, starting_width
maze[starting_height][starting_width] = "X" 
print_maze(maze)

1
u u u u u u u u u u 

u u X u u u u u c u 

u u u u u u u u u u 

u u u u u u u u u u 



In [279]:
maze_generation_start_height = randint(1, height - 1)
maze_generation_start_width = randint(1, width - 1)

maze_generation_start_height, maze_generation_start_width

(1, 2)

In [281]:
# So now we will mark this block as a path and add the surrounding walls to the walls list:

maze[starting_height][starting_width] = cell

walls = []
walls.append([starting_height-1, starting_width])
walls.append([starting_height, starting_width-1])
walls.append([starting_height, starting_width+1])
walls.append([starting_height+1, starting_width])
print_maze(maze)

u u u u u u u u u u 

u u u u u u u u c u 

u u u u u u u u u u 

u u u u u u u u u u 



In [180]:
walls

[[3, 7], [4, 6], [4, 8], [5, 7]]

In [181]:
# Note: Do this instead, why do the following??
for w in walls:
    maze[w[0]][w[1]] = "w"
print_maze(maze)

# In order to complete step two, we need to denote the blocks round the starting cell as walls:
# maze[starting_height-1][starting_width] = wall
# maze[starting_height][starting_width-1] = wall
# maze[starting_height][starting_width+1] = wall
# maze[starting_height+1][starting_width] = wall
# print_maze(maze)

u u u u u u u u u u 

u u u u u u u u u u 

u u u u u u u u u u 

u u u u u u u w u u 

u u u u u u w c w u 

u u u u u u u w u u 

u u u u u u u u u u 

u u u u u u u u u u 

u u u u u u u u u u 

u u u u u u u u u u 



We have two possibilities here.
First we need to check the blocks to the left and right of the wall we are processing and then we need to check the blocks above and below the wall. Let’s visualize it in order to understand it:

Case 1 (we check the blocks at left and right of the selected wall):

|   | u |   |
|---|---|---|
| u | w | c |
|   | u |   |

Case 2 (we check the blocks above and below the selected wall):

|   | u |   |
|---|---|---|
| u | w | u |
|   | c |   |

Then provide the mirrored conditions for both, so 4 cases to check.


> Make the wall a passage and mark the unvisited cell as part of the maze

This part is a little bit tricky. We need to make sure that every wall that we are going to turn into passage, does not have more than one cell around it. If we do not make this kind of check now, we will end up with a maze that has clusters of passages and the maze will not be good. So we will add an extra check if the surrounding cells are less than two. But first we need to create a function that checks that:


In [182]:
def surroundingCells(rand_wall):
    s_cells = 0
    if (maze[rand_wall[0]-1][rand_wall[1]] == 'c'):
        s_cells += 1
    if (maze[rand_wall[0]+1][rand_wall[1]] == 'c'):
        s_cells += 1
    if (maze[rand_wall[0]][rand_wall[1]-1] == 'c'):
        s_cells +=1
    if (maze[rand_wall[0]][rand_wall[1]+1] == 'c'):
        s_cells += 1
    return s_cells


# Finally, we need to remove this processed block from the walls list. Then we need to continue with the next iteration. 
def delete_wall(rand_wall):
    for wall in walls:
        if (wall[0] == rand_wall[0] and wall[1] == rand_wall[1]):
            walls.remove(wall)

In [183]:
# While there are walls in the list pick a random wall from the list
while walls:
    # Pick a random wall
    # Why this?? Gross
    # rand_wall = walls[int(random.random()*len(walls))-1]
    rand_wall = walls[randint(0, len(walls) - 1)]

    # We need to make sure we are always accessing a correct index.
    # Meaning that if the selected wall is the one on the first line of the maze,
    # we cannot go and check the cell above it, since it would create an IndexError in Python.
    # Then:
    # If only one of the two cells that the wall divides is visited

    # Check if it is a left wall
    if rand_wall[1] != 0:
        if (
            maze[rand_wall[0]][rand_wall[1] - 1] == "u"
            and maze[rand_wall[0]][rand_wall[1] + 1] == "c"
        ):
            # Find the number of surrounding cells
            s_cells = surroundingCells(rand_wall)

            if s_cells < 2:
                # Denote the new path
                maze[rand_wall[0]][rand_wall[1]] = "c"

                # Mark the new walls
                # Upper cell
                if rand_wall[0] != 0:
                    if maze[rand_wall[0] - 1][rand_wall[1]] != "c":
                        maze[rand_wall[0] - 1][rand_wall[1]] = "w"
                    if [rand_wall[0] - 1, rand_wall[1]] not in walls:
                        walls.append([rand_wall[0] - 1, rand_wall[1]])

                # Bottom cell
                if rand_wall[0] != height - 1:
                    if maze[rand_wall[0] + 1][rand_wall[1]] != "c":
                        maze[rand_wall[0] + 1][rand_wall[1]] = "w"
                    if [rand_wall[0] + 1, rand_wall[1]] not in walls:
                        walls.append([rand_wall[0] + 1, rand_wall[1]])

                # Leftmost cell
                if rand_wall[1] != 0:
                    if maze[rand_wall[0]][rand_wall[1] - 1] != "c":
                        maze[rand_wall[0]][rand_wall[1] - 1] = "w"
                    if [rand_wall[0], rand_wall[1] - 1] not in walls:
                        walls.append([rand_wall[0], rand_wall[1] - 1])

            # Delete wall
            for wall in walls:
                if wall[0] == rand_wall[0] and wall[1] == rand_wall[1]:
                    walls.remove(wall)

            continue

    # Check if it is an upper wall
    if rand_wall[0] != 0:
        if (
            maze[rand_wall[0] - 1][rand_wall[1]] == "u"
            and maze[rand_wall[0] + 1][rand_wall[1]] == "c"
        ):

            s_cells = surroundingCells(rand_wall)
            if s_cells < 2:
                # Denote the new path
                maze[rand_wall[0]][rand_wall[1]] = "c"

                # Mark the new walls
                # Upper cell
                if rand_wall[0] != 0:
                    if maze[rand_wall[0] - 1][rand_wall[1]] != "c":
                        maze[rand_wall[0] - 1][rand_wall[1]] = "w"
                    if [rand_wall[0] - 1, rand_wall[1]] not in walls:
                        walls.append([rand_wall[0] - 1, rand_wall[1]])

                # Leftmost cell
                if rand_wall[1] != 0:
                    if maze[rand_wall[0]][rand_wall[1] - 1] != "c":
                        maze[rand_wall[0]][rand_wall[1] - 1] = "w"
                    if [rand_wall[0], rand_wall[1] - 1] not in walls:
                        walls.append([rand_wall[0], rand_wall[1] - 1])

                # Rightmost cell
                if rand_wall[1] != width - 1:
                    if maze[rand_wall[0]][rand_wall[1] + 1] != "c":
                        maze[rand_wall[0]][rand_wall[1] + 1] = "w"
                    if [rand_wall[0], rand_wall[1] + 1] not in walls:
                        walls.append([rand_wall[0], rand_wall[1] + 1])

            # Delete wall
            for wall in walls:
                if wall[0] == rand_wall[0] and wall[1] == rand_wall[1]:
                    walls.remove(wall)

            continue

    # Check the bottom wall
    if rand_wall[0] != height - 1:
        if (
            maze[rand_wall[0] + 1][rand_wall[1]] == "u"
            and maze[rand_wall[0] - 1][rand_wall[1]] == "c"
        ):

            s_cells = surroundingCells(rand_wall)
            if s_cells < 2:
                # Denote the new path
                maze[rand_wall[0]][rand_wall[1]] = "c"

                # Mark the new walls
                if rand_wall[0] != height - 1:
                    if maze[rand_wall[0] + 1][rand_wall[1]] != "c":
                        maze[rand_wall[0] + 1][rand_wall[1]] = "w"
                    if [rand_wall[0] + 1, rand_wall[1]] not in walls:
                        walls.append([rand_wall[0] + 1, rand_wall[1]])
                if rand_wall[1] != 0:
                    if maze[rand_wall[0]][rand_wall[1] - 1] != "c":
                        maze[rand_wall[0]][rand_wall[1] - 1] = "w"
                    if [rand_wall[0], rand_wall[1] - 1] not in walls:
                        walls.append([rand_wall[0], rand_wall[1] - 1])
                if rand_wall[1] != width - 1:
                    if maze[rand_wall[0]][rand_wall[1] + 1] != "c":
                        maze[rand_wall[0]][rand_wall[1] + 1] = "w"
                    if [rand_wall[0], rand_wall[1] + 1] not in walls:
                        walls.append([rand_wall[0], rand_wall[1] + 1])

            # Delete wall
            for wall in walls:
                if wall[0] == rand_wall[0] and wall[1] == rand_wall[1]:
                    walls.remove(wall)

            continue

    # Check the right wall
    if rand_wall[1] != width - 1:
        if (
            maze[rand_wall[0]][rand_wall[1] + 1] == "u"
            and maze[rand_wall[0]][rand_wall[1] - 1] == "c"
        ):

            s_cells = surroundingCells(rand_wall)
            if s_cells < 2:
                # Denote the new path
                maze[rand_wall[0]][rand_wall[1]] = "c"

                # Mark the new walls
                if rand_wall[1] != width - 1:
                    if maze[rand_wall[0]][rand_wall[1] + 1] != "c":
                        maze[rand_wall[0]][rand_wall[1] + 1] = "w"
                    if [rand_wall[0], rand_wall[1] + 1] not in walls:
                        walls.append([rand_wall[0], rand_wall[1] + 1])
                if rand_wall[0] != height - 1:
                    if maze[rand_wall[0] + 1][rand_wall[1]] != "c":
                        maze[rand_wall[0] + 1][rand_wall[1]] = "w"
                    if [rand_wall[0] + 1, rand_wall[1]] not in walls:
                        walls.append([rand_wall[0] + 1, rand_wall[1]])
                if rand_wall[0] != 0:
                    if maze[rand_wall[0] - 1][rand_wall[1]] != "c":
                        maze[rand_wall[0] - 1][rand_wall[1]] = "w"
                    if [rand_wall[0] - 1, rand_wall[1]] not in walls:
                        walls.append([rand_wall[0] - 1, rand_wall[1]])

            # Delete wall
            for wall in walls:
                if wall[0] == rand_wall[0] and wall[1] == rand_wall[1]:
                    walls.remove(wall)

            continue

    # Delete the wall from the list anyway
    for wall in walls:
        if wall[0] == rand_wall[0] and wall[1] == rand_wall[1]:
            walls.remove(wall)


In [184]:
print_maze(maze)

u w u w u w u u w u 

w c w c w c w w c w 

w c c c w c w w c w 

w c w c c c c w c w 

w c w c w w c c c w 

w c w c w c c w c w 

u w w c w w c w w u 

w c c c w c c c c w 

w c w c w w c w c w 

u w u w u u w u w u 



In [186]:
# If you print the maze when the list of walls is empty, 
# you will notice that there are some cells that will be unvisited. 
# You need to make them walls:
def make_walls(width, height):
    for i in range(0, height):
        for j in range(0, width):
            if (maze[i][j] == 'u'):
                maze[i][j] = 'w'

# m = init_maze(width, height)
# print_maze(m)
make_walls(width, height)
print_maze(maze)

w w w w w w w w w w 

w c w c w c w w c w 

w c c c w c w w c w 

w c w c c c c w c w 

w c w c w w c c c w 

w c w c w c c w c w 

w w w c w w c w w w 

w c c c w c c c c w 

w c w c w w c w c w 

w w w w w w w w w w 



In [154]:
def create_entrance_exit(width, height):
    for i in range(0, width):
        i = 2
        if (maze[1][i] == 'c'):
            maze[0][i] = 'c'
            break
    for i in range(width-1, 0, -1):
        i = 2
        if (maze[height-2][i] == 'c'):
            maze[height-1][i] = 'c'
            break

In [155]:
init_maze(width, height)
make_walls(width, height)
create_entrance_exit(width, height)
print_maze(maze)

w w c w w w w w w w 

w c c c c c c c c w 

w c w c w w c w w w 

w c w c c w c c c w 

w c w c w w w w w w 

w w w c w c w w c w 

w w c c c c c c c w 

w c c w w w c w c w 

w w c c c w c w c w 

w w c w w w w w w w 

